# Knowledge Graphs - datasets quality review
### Analysis of the quality of top publishers at data.europa.eu
##### Patryk Rakus, Michał Tomczyk

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from utils import SPARQL_to_df
from FAIR import findability, accessibility, interoperability, reusability
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
connection = SPARQLWrapper("https://data.europa.eu/sparql")

In [3]:
PREFIXES = """
    PREFIX dcat: <http://www.w3.org/ns/dcat#>
    PREFIX odp:  <http://data.europa.eu/euodp/ontologies/ec-odp#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX void: <http://rdfs.org/ns/void#>
"""


Finding the top themes:

In [4]:
query = '''
SELECT ?theme (count(*) as ?DatasetsPublished) WHERE {
    ?datasetURI a dcat:Dataset;
    dcat:theme ?theme .
}
GROUP BY ?theme
ORDER BY DESC(?DatasetsPublished)
'''

In [5]:
df = SPARQL_to_df(connection, query, PREFIXES)

In [65]:
df.iloc[10: 20]

,theme,DatasetsPublished
10,http://publications.europa.eu/resource/authori...,57074
11,http://publications.europa.eu/resource/authori...,35656
12,http://inspire.ec.europa.eu/theme/cp,27513
13,http://eurovoc.europa.eu/5079,27511
14,http://eurovoc.europa.eu/100142,26727
15,http://eurovoc.europa.eu/100154,26154
16,http://publications.europa.eu/resource/authori...,26111
17,http://eurovoc.europa.eu/100160,25726
18,http://publications.europa.eu/resource/authori...,14263
19,http://eurovoc.europa.eu/100156,11005


In [66]:
df.iloc[10:20]['DatasetsPublished'].astype('Int64').sum()

np.int64(277740)

In [71]:
top_themes = list(df.iloc[10:20]['theme'])

In [72]:
newline = '\n'

In [73]:
query = f'''
VALUES ?theme {{
    {"".join(f"<{theme}>" + newline for theme in top_themes)}
    }}
    ?dataset a dcat:Dataset ;
           dcat:theme ?theme .
'''

In [74]:
df_findability = findability.findability_aggregated_properties(connection, query, PREFIXES, 'theme')

In [75]:
df_findability

,theme,totalDatasets,avgKeywords,percentageWithTitle,percentageWithIssuedDate,percentageWithLocation,percentageWithType,percentageWithPartOf
0,http://publications.europa.eu/resource/authori...,57074,1,100,44.102393384027753,90.601675018397169,71.23734099590006,45.565406314609104
1,http://publications.europa.eu/resource/authori...,26111,1,100,67.998161694305082,59.515146872965417,25.7018114970702,2.17532840565279
2,http://eurovoc.europa.eu/100156,11005,1,100,39.054975011358473,100,99.772830531576556,25.615629259427533
3,http://eurovoc.europa.eu/5079,27511,1,100,0.574315728254153,100,99.920031987205118,47.944458580204282
4,http://inspire.ec.europa.eu/theme/cp,27513,1,100,0.599716497655654,99.989096063678988,99.938211027514266,47.940973358048922
5,http://publications.europa.eu/resource/authori...,35656,1,100,72.79279784608481,66.249719542293022,5.056652456809513,0.098160197442226
6,http://eurovoc.europa.eu/100142,26727,1,99.996258465222434,3.303775208590564,99.988775395667303,99.921427769671119,93.5533355782542
7,http://eurovoc.europa.eu/100160,25726,1,100,2.483868459923812,100,99.968903055274819,96.116769027443054
8,http://publications.europa.eu/resource/authori...,14263,1,100,99.326929818411274,99.97195540910047,0.02804459089953,0
9,http://eurovoc.europa.eu/100154,26154,1,100,2.840865641966812,100,99.927353368509597,94.845912671101935


In [76]:
df_interoperability = interoperability.interoperability_aggregated_properties(connection, query, PREFIXES, 'theme')

In [77]:
df_interoperability

,theme,totalDatasets,avgReferences,percentageWithLanguage,percentageWithAccessRights,percentageBeingVersionOf,percentageWithIdentifier,percentageWithRightsHolder
0,http://publications.europa.eu/resource/authori...,26111,1.030829918425185,64.781126728198843,32.936310367278159,8.123013289418253,86.193558270460725,3.967676458197694
1,http://publications.europa.eu/resource/authori...,57074,1.011896835687003,83.806987419840908,19.646423940848723,0.417002487998038,94.782212566142201,8.152573851491047
2,http://eurovoc.europa.eu/100156,11005,1,98.237164925034075,0,0,98.882326215356656,37.255792821444798
3,http://publications.europa.eu/resource/authori...,35656,1.069581557101189,75.68712138209558,38.018846757908907,9.142921247475881,89.395894099169845,9.586044424500785
4,http://eurovoc.europa.eu/100142,26727,1,99.711901822127437,0,0,99.363939087813821,2.660231226849254
5,http://inspire.ec.europa.eu/theme/cp,27513,1,99.981826772798314,0,0,99.898229927670556,4.136226511103842
6,http://eurovoc.europa.eu/5079,27511,1,99.963650903275054,0.018174548362473,0,99.89822252917015,4.118352658936425
7,http://eurovoc.europa.eu/100160,25726,1,99.840628158283449,0,0,99.906709165824458,1.492653346808676
8,http://publications.europa.eu/resource/authori...,14263,1,64.972305966486714,99.593353431956811,0,99.97195540910047,99.144639977564327
9,http://eurovoc.europa.eu/100154,26154,1,99.904412326986312,0,0,99.889118299304122,1.953812036399786


In [78]:
df_both_attributes = df_interoperability.merge(df_findability, on='theme')

In [79]:
df_both_attributes

,theme,totalDatasets_x,avgReferences,percentageWithLanguage,percentageWithAccessRights,percentageBeingVersionOf,percentageWithIdentifier,percentageWithRightsHolder,totalDatasets_y,avgKeywords,percentageWithTitle,percentageWithIssuedDate,percentageWithLocation,percentageWithType,percentageWithPartOf
0,http://publications.europa.eu/resource/authori...,26111,1.030829918425185,64.781126728198843,32.936310367278159,8.123013289418253,86.193558270460725,3.967676458197694,26111,1,100,67.998161694305082,59.515146872965417,25.7018114970702,2.17532840565279
1,http://publications.europa.eu/resource/authori...,57074,1.011896835687003,83.806987419840908,19.646423940848723,0.417002487998038,94.782212566142201,8.152573851491047,57074,1,100,44.102393384027753,90.601675018397169,71.23734099590006,45.565406314609104
2,http://eurovoc.europa.eu/100156,11005,1,98.237164925034075,0,0,98.882326215356656,37.255792821444798,11005,1,100,39.054975011358473,100,99.772830531576556,25.615629259427533
3,http://publications.europa.eu/resource/authori...,35656,1.069581557101189,75.68712138209558,38.018846757908907,9.142921247475881,89.395894099169845,9.586044424500785,35656,1,100,72.79279784608481,66.249719542293022,5.056652456809513,0.098160197442226
4,http://eurovoc.europa.eu/100142,26727,1,99.711901822127437,0,0,99.363939087813821,2.660231226849254,26727,1,99.996258465222434,3.303775208590564,99.988775395667303,99.921427769671119,93.5533355782542
5,http://inspire.ec.europa.eu/theme/cp,27513,1,99.981826772798314,0,0,99.898229927670556,4.136226511103842,27513,1,100,0.599716497655654,99.989096063678988,99.938211027514266,47.940973358048922
6,http://eurovoc.europa.eu/5079,27511,1,99.963650903275054,0.018174548362473,0,99.89822252917015,4.118352658936425,27511,1,100,0.574315728254153,100,99.920031987205118,47.944458580204282
7,http://eurovoc.europa.eu/100160,25726,1,99.840628158283449,0,0,99.906709165824458,1.492653346808676,25726,1,100,2.483868459923812,100,99.968903055274819,96.116769027443054
8,http://publications.europa.eu/resource/authori...,14263,1,64.972305966486714,99.593353431956811,0,99.97195540910047,99.144639977564327,14263,1,100,99.326929818411274,99.97195540910047,0.02804459089953,0
9,http://eurovoc.europa.eu/100154,26154,1,99.904412326986312,0,0,99.889118299304122,1.953812036399786,26154,1,100,2.840865641966812,100,99.927353368509597,94.845912671101935
